# 🧠 Business Optimization using Linear Programming & PuLP

This project solves a product mix optimization problem for a furniture manufacturing company using Python and PuLP. It includes visualization, slack analysis, and exportable data.

In [ ]:
!pip install pulp --quiet
import matplotlib.pyplot as plt
from pulp import LpMaximize, LpProblem, LpVariable, LpStatus, value
import pandas as pd

data = pd.DataFrame({
    'Product': ['Chair', 'Table', 'Desk'],
    'Machine_A_Hours': [2, 3, 4],
    'Machine_B_Hours': [3, 2, 5],
    'Labor_Hours': [2, 2, 3],
    'Profit': [150, 200, 300]
})

resources = {
    'Machine_A': 150,
    'Machine_B': 120,
    'Labor': 100
}

model = LpProblem("Furniture_Production_Optimization", LpMaximize)
products = data['Product']
product_vars = {product: LpVariable(f"{product}", lowBound=0, cat='Integer') for product in products}

model += sum(product_vars[p] * data.loc[data['Product'] == p, 'Profit'].values[0] for p in products), "Total_Profit"
model += sum(product_vars[p] * data.loc[data['Product'] == p, 'Machine_A_Hours'].values[0] for p in products) <= resources['Machine_A'], "Machine_A_Constraint"
model += sum(product_vars[p] * data.loc[data['Product'] == p, 'Machine_B_Hours'].values[0] for p in products) <= resources['Machine_B'], "Machine_B_Constraint"
model += sum(product_vars[p] * data.loc[data['Product'] == p, 'Labor_Hours'].values[0] for p in products) <= resources['Labor'], "Labor_Constraint"

model.solve()

print("🔍 Status:", LpStatus[model.status])
for p in products:
    print(f"Produce {product_vars[p].varValue} units of {p}")
print("💰 Maximum Profit: ₹", value(model.objective))

summary = pd.DataFrame({
    'Product': products,
    'Units_Produced': [product_vars[p].varValue for p in products],
    'Profit_Per_Unit': data['Profit'],
    'Total_Profit': [product_vars[p].varValue * data.loc[data['Product'] == p, 'Profit'].values[0] for p in products]
})
summary.to_csv("production_summary.csv", index=False)

slack = {
    "Machine_A_Slack": resources['Machine_A'] - sum(product_vars[p].varValue * data.loc[data['Product'] == p, 'Machine_A_Hours'].values[0] for p in products),
    "Machine_B_Slack": resources['Machine_B'] - sum(product_vars[p].varValue * data.loc[data['Product'] == p, 'Machine_B_Hours'].values[0] for p in products),
    "Labor_Slack": resources['Labor'] - sum(product_vars[p].varValue * data.loc[data['Product'] == p, 'Labor_Hours'].values[0] for p in products)
}
slack_df = pd.DataFrame(slack.items(), columns=["Resource", "Unused_Hours"])
slack_df.to_csv("slack_report.csv", index=False)

plt.figure(figsize=(8, 5))
plt.bar(summary['Product'], summary['Units_Produced'], color='teal')
plt.title("Optimal Units to Produce")
plt.xlabel("Product")
plt.ylabel("Units")
plt.grid(axis='y')
plt.savefig("production_plan.png")
plt.show()